In [1]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Layer

# Attention Layer
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(
            name='attention_weight',
            shape=(input_shape[-1], 1),
            initializer='glorot_uniform',
            trainable=True
        )
        super(Attention, self).build(input_shape)

    def call(self, inputs):
        score = tf.matmul(inputs, self.W)
        score = tf.nn.softmax(score, axis=1)
        context_vector = inputs * score
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

2025-04-11 11:44:31.550777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744371871.740643      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744371871.797126      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from tensorflow.keras.models import load_model

model = load_model('/kaggle/input/bilstm/tensorflow2/default/1/bilstm_best_model.h5', custom_objects={'Attention': Attention})
print("model load successfully")


I0000 00:00:1744371884.014458      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


model load successfully


In [5]:
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

# load tokenizer
with open('/kaggle/input/tokenizer/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# get test data
test_df = pd.read_csv('/kaggle/input/nli-test/test.csv')
test_texts_raw = test_df['premise'] + ' [SEP] ' + test_df['hypothesis']

# clean
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s.,!?]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

test_texts = [clean_text(t) for t in test_texts_raw]
X_test_seq = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(X_test_seq, maxlen=70, padding='post', truncating='post')


In [8]:
import numpy as np
import pandas as pd

pred_probs = model.predict(X_test)

pred_labels = np.argmax(pred_probs, axis=1)

# save to CSV
pd.DataFrame(pred_labels, columns=["prediction"]).to_csv('prediction666.csv', index=False)

print("✅ prediction.csv with header saved!")


104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
✅ prediction.csv with header saved!
